# Background

## Data Description

Data Description:

The data comes from direct marketing efforts of a European banking institution. The marketing campaign involves making a phone call to a customer, often multiple times to ensure a product subscription, in this case a term deposit. Term deposits are usually short-term deposits with maturities ranging from one month to a few years. The customer must understand when buying a term deposit that they can withdraw their funds only after the term ends. All customer information that might reveal personal information is removed due to privacy concerns.


### Attributes:

| Column	|Description|
|-|-|
|age	|Age of customer (numeric)|
|job	|Type of job (categorical)|
|marital	|Marital status (categorical)|
|education	|Education level (categorical)|
|default	|Has credit in default? (binary)|
|balance	|Average yearly balance, in euros (numeric)|
|housing	|Has a housing loan? (binary)|
|loan	|Has personal loan? (binary)|
|contact	|Contact communication type (categorical)|
|day	|Last contact day of the month (numeric)|
|month	|Last contact month of the year (categorical)|
|duration	|Last contact duration, in seconds (numeric)|
|campaign	|Number of contacts performed during this campaign and for this client (numeric, includes last contact)|

### Output (desired target):

$y$ - has the client subscribed to a term deposit? (binary)

### Goal(s):

- Predict if the customer will subscribe (yes/no) to a term deposit (variable y)
- Find customers who are more likely to buy the investment product. Determine the segment(s) of customers our client should prioritize.
- What makes the customers buy? 


### Success Metric(s):

Hit %81 or above accuracy by evaluating with 5-fold cross validation and reporting the average performance score.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
import seaborn as sns

In [2]:
csv_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw', 'term-deposit-marketing-2020.csv'))

## Exploratory data analysis using, JupySQL, SQL queries and DuckDB

**Note**

The queries below asume the usage of JupySQL and Jupyter notebooks.

Ensure the following are installed

```
duckdb-engine
jupysql
```

In [3]:
%load_ext sql
%sql duckdb:///:memory:

In [4]:
query = f"""
CREATE TABLE term_deposit_marketing AS SELECT * FROM read_csv_auto('{csv_path}')
"""

%sql {{query}}


*  duckdb:///:memory:
Done.


Count
40000


In [5]:
f5 = """
    SELECT * FROM term_deposit_marketing LIMIT 5;
"""

%sql {{f5}}

*  duckdb:///:memory:
Done.


age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,no
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,no
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,no
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,no
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,no


## Building the model

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score

In [7]:
full_data = %sql SELECT * FROM term_deposit_marketing;
data = full_data.DataFrame()


*  duckdb:///:memory:
Done.


Preprocess the data

In [8]:
# Prepare target variable
data['y'] = data['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Define feature columns and target column
X = data.drop(columns=['y'])
y = data['y']

Split the data into training and testing datasets

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Create a preprocessing pipeline

In [10]:
# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(exclude=['object']).columns.tolist()

# Define transformers for categorical and numerical columns
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

Create a pipeline and fit the model

In [11]:
# Create a pipeline with preprocessing and RandomForestClassifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])

# Perform 5-fold cross-validation
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')

# Calculate the average accuracy
avg_accuracy = cv_scores.mean()

print("5-fold Cross-validation Accuracy:", avg_accuracy)

success_metric = 0.81

if avg_accuracy >= success_metric:
    print("Success: The average accuracy is above or equal to the success metric.")
else:
    print("Failure: The average accuracy is below the success metric.")


Make predictions and evaluate

In [ ]:
# Fit the model
pipeline.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
